# XGBOOST 2020, 2021, 2022

In [2]:
# Deze code is al gerund, dus opnieuw runnen heeft geen zin.

# def append_str_to_columns(df, str, exclude_columns):
#     new_columns = []
#     for column in df.columns:
#         if column not in exclude_columns:
#             new_columns.append(column[:7] + '{}'.format(str))
#         else:
#             new_columns.append(column)
#     df.columns = new_columns
#     return df

# stringlist = ["_2019","_2020"]
# dflist = ["./data/kwb-2019.xlsx","./data/kwb-2020.xls"]

# exclude_columns = ['gwb_code_10', 'gwb_code_8', 'regio', 'gm_naam', 'recs', 'gwb_code']

# for i in range(len(stringlist)):
#     df = append_str_to_columns(pd.read_excel(dflist[i]), stringlist[i], exclude_columns)
#     savestring = '{}.csv'.format(dflist[i][:15])
#     df.to_csv(savestring, index=False)

# Dit is ook uitgevoerd dus hoeft niet nog een keer.

# zorg21 = zorg21[zorg21['VormenVanJeugdzorg'] == 'T001203']
# zorg21 = zorg21[zorg21['Wijken'].str[:2] != 'BU']
# kern21 = pd.read_excel('./data/kwb-2021.xls')
# kern21 = kern21.applymap(lambda x: x.strip() if isinstance(x, str) else x)
# kern21 = kern21[kern21['gwb_code_10'].str[:2] != 'BU']
# df = kern21.merge(zorg21, left_on='gwb_code_10', right_on='Wijken', how='left')
# df = df.replace('.', np.nan)


# df = pd.read_excel("./data/kwb-2022.xls")
# zorg = pd.read_csv('./data/zorg22.csv')
# zorg = zorg[['Wijken', 'p_jz_tn']]

# df = df.drop('p_jz_tn', axis=1)
# df = pd.merge(df, zorg, left_on='gwb_code_10', right_on='Wijken', how='left')
# df.to_csv("./data/kwb-2022.csv", index=False)

NameError: name 'pd' is not defined

In [3]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

kwb_20 = pd.read_excel('./data/kwb-2020.xls')
zorg21 = pd.read_csv('./data/zorg21.csv')
zorg22 = pd.read_csv('./data/zorg22.csv')
kwb_20 = kwb_20[['gwb_code_10', 'p_jz_tn']]
zorg21 = zorg21[['Wijken', 'p_jz_tn']]
zorg22 = zorg22[['Wijken', 'p_jz_tn']]
df = pd.merge(kwb_20, zorg21, left_on='gwb_code_10', right_on='Wijken', how='inner')
df = pd.merge(df, zorg22, on='Wijken', how='inner')
df.columns = ['gwb_code_10', 'p_jz_tn_20', 'Wijken', 'p_jz_tn_21', 'p_jz_tn_22']

In [4]:
# List of columns to exclude from merging
exclude_columns = ['gwb_code_8', 'regio', 'gm_naam', 'recs', 'gwb_code']

# Load and combine datasets
dfs = []
for year in range(2013, 2023):
    filename = f'./data/kwb-{year}.csv'
    df = pd.read_csv(filename)
    if year == 2013:
        combined_df = df
    else:
        combined_df = pd.merge(combined_df, df, on='gwb_code_10', how='outer', suffixes=('', f'_{year}'))

# Merge datasets based on 'gwb_code_10'
merged_df = combined_df.replace([np.inf, -np.inf, np.nan], np.nan)
merged_df = combined_df.fillna(combined_df.mean())
merged_df = merged_df.drop(exclude_columns, axis=1)

for col in merged_df.columns:
    try:
        merged_df[col] = merged_df[col].astype(float)
    except ValueError:
        merged_df = merged_df.drop(col, axis=1)

print(merged_df.columns)

# Extract features and target variable
target = merged_df['p_jz_tn_y_2022']
features = merged_df.drop('p_jz_tn_y_2022', axis=1)

C:\Users\stanf\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (106,107) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\stanf\AppData\Local\Temp/ipykernel_16720/2915219861.py:16: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  merged_df = combined_df.fillna(combined_df.mean())


Index(['a_inw_2013', 'a_man_2013', 'a_vrouw_2013', 'a_geb_2013', 'a_ste_2013',
       'a_hh_2013', 'a_woning_2013', 'a_pau_2013', 'a_lftj6j_2013',
       'a_lfto6j_2013',
       ...
       'Huisarts_11_2022', 'Jeugdarts_12_2022',
       'GecertificeerdeInstelling_13_2022', 'MedischSpecialist_14_2022',
       'RechterOfficierVanJustitie_15_2022', 'GeenVerwijzer_16_2022',
       'VerwijzerOnbekend_17_2022', 'IndelingswijzigingWijken_21_2022',
       'a_00_24_2022', 'p_jz_tn_y_2022'],
      dtype='object', length=524)


In [5]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Train XGBoost model
xgb_model = xgb.XGBRegressor()
xgb_model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = xgb_model.predict(X_test)

# Calculate evaluation metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R^2 Score:", r2)

Mean Squared Error: 0.5749127657914481
R^2 Score: 0.8563592789189866


In [6]:
predictions_2023 = xgb_model.predict(features)

# Optional: Save predictions to a CSV file
predictions_2023_df = pd.DataFrame({'p_jz_tn_2023': predictions_2023})
predictions_2023_df.to_csv('predictions_2023.csv', index=False)